In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os

ROOT_DIR = os.path.abspath('..')
sys.path.append(ROOT_DIR)
os.chdir(ROOT_DIR)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from neuron.neurons import User, Neuron, ConversationalOrchestrator, ConversationalOrchestratorManager
import os

c:\Users\FS-Ma\OneDrive\Documents\projects\neuron\.venv\Lib\site-packages\accelerate\utils\other.py:220: DeprecationWarning: numpy.core is deprecated and has been renamed to numpy._core. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.multiarray.
  np.core.multiarray._reconstruct,


In [7]:
llm_config = {
    "config_list": [
        {
            "client": "groq",
            "temperature": 0.0,
            "model": "llama3-70b-8192",
            "api_key": os.getenv("GROQ_API_KEY")
        }
    ]
}

user = User(
    description="You are a human user interacting with conversational and recommendation agents."
)

conversational = Neuron(
    name="conversational",
    llm_config=llm_config,
    system_message="""
    You are a friendly Conversation AI.
    Your job is to handle all general interactions naturally and pleasantly.
    Focus on keeping the conversation flowing unless a specialist (like the Recommender) is needed.
""",
    description="This agent acts as a social companion and facilitates interactions."
)

llm_config_2 = {
    "config_list": [
        {
            "client": "groq",
            "temperature": 0.0,
            "model": "llama3-70b-8192",
            "api_key": os.getenv("GROQ_API_KEY")
        }
    ]
}

recommendation = Neuron(
    name="recommender",
    llm_config=llm_config_2,
    system_message="""
    You are a specialized Recommender AI.
    Your role is to respond directly to user requests for recommendations.

When the user asks for recommendations (such as suggesting products, services, options, or anything similar), you must respond clearly and directly, providing a concise list or a short explanation.

- Never decline a recommendation request.
- Keep your responses focused only on recommending options.
- Do not ask questions unless necessary for a better recommendation.
- Stay friendly and helpful.

Examples of valid requests you should handle:
- "Can you recommend a good restaurant?"
- "Suggest a book to read."
- "What laptop should I buy?"

Always take the initiative to recommend something relevant, even if the user's question is vague.

""",
    description="This agent specializes in offering product recommendations."
)

chitchat = ConversationalOrchestrator(
    agents=[user, conversational, recommendation],
    messages=[],
    max_round=100,
    speaker_selection_method="auto",
    #send_introductions=True,
    allow_repeat_speaker = False,
)

chitchat_manager = ConversationalOrchestratorManager(chitchat=chitchat, llm_config=llm_config)

In [8]:
chat_result = user.initiate_chat(
    chitchat_manager, message="Oi tudo bem?",
    cache=None
)

user ⟶ conversational_orchestrator:
Oi tudo bem?

Next speaker: conversational

conversational ⟶ conversational_orchestrator:
Olá! Tudo bem, obrigado! E você? Como foi seu dia?

Next speaker: user

user ⟶ conversational_orchestrator:
Estou bem, obrigado

Next speaker: conversational

conversational ⟶ conversational_orchestrator:
Que bom! É sempre uma boa notícia quando as coisas vão bem. Você tem planos para o fim de semana ou vai relaxar um pouco?

Next speaker: user

user ⟶ conversational_orchestrator:
Gostaria de recomendações

Next speaker: recommender

recommender ⟶ conversational_orchestrator:
Excelente escolha!

Se você está procurando por algo para fazer no fim de semana, aqui vão algumas recomendações:

* Filme: "Everything Everywhere All at Once" (uma comédia dramática muito elogiada)
* Série: "Stranger Things" (uma série de suspense e aventura muito popular)
* Livro: "The Hitchhiker's Guide to the Galaxy" (um clássico de ficção científica divertido)
* Jogo: "Portal" (um jogo